# LlamaIndex RAG with Ollama (gemma:2b)

Build a Retrieval-Augmented Generation (RAG) pipeline using local components: LlamaIndex for indexing, Ollama (`gemma:2b`) for LLM inference, and Weaviate as the vector store.

For detailed setup, architecture, and troubleshooting, see `CHAPTER3_BLOG.md` in this folder.

## Prerequisites

- Python 3.10+ with virtual environment
- Ollama at `http://localhost:11434` with `gemma:2b` available
- Weaviate at `http://localhost:8080` (run: `docker run -d -p 8080:8080 -p 50051:50051 semitechnologies/weaviate:latest`)

Check health:
```bash
curl http://localhost:11434/api/show
curl http://localhost:8080/v1/
```

## Quick Start

1. Ensure Ollama and Weaviate are running.
2. Place documents in `./docs/` (or run the document summarizer to extract them).
3. Run notebook cells in order: imports → LLM config → Weaviate connection → load docs → index → query.
4. Use `ask_question(...)` to run queries.

## Troubleshooting & Tips

- ReadTimeouts: increase `keep_alive` and the HTTP timeout in the Ollama client; add retries/backoff around `query_engine.query` calls.
- Slow embedding/indexing: use smaller batch sizes or a faster embedding model during development.
- Weaviate schema issues: persist schema and avoid re-creating classes on every run; inspect Weaviate logs for errors.
- Logs: enable verbose prints around the LLM call and Weaviate requests to capture failing payloads.

If you'd like, I can add automatic retry logic for the Ollama calls and incorporate an exponential backoff helper in the notebook.

In [ ]:
# Install required packages
!pip3 install llama-index llama-index-llms-azure-openai llama-index-embeddings-ollama llama-index-vector-stores-weaviate llama-index-llms-ollama weaviate-client

In [ ]:
# Import necessary libraries
import os
from pathlib import Path
from dotenv import load_dotenv

# LlamaIndex imports
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore


# Weaviate client
import weaviate

# Load environment variables
load_dotenv()

print("All imports successful")

In [ ]:


# Configure Ollama LLM for LlamaIndex
llm = Ollama(
    model="gemma:2b",
    base_url="http://localhost:11434",
    temperature=0.1,
    keep_alive=120,
    # optional: pass other ollama-specific kwargs if needed:
    # ollama_kwargs={"n_predict": 1024}
)

print("Ollama AI LLM configured")

# Configure Ollama embeddings
embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

print("Ollama embeddings configured")

# Set global defaults for LlamaIndex
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

print("\nLlamaIndex settings configured"

In [ ]:
# Connect to Weaviate vector store
print("🔌 Connecting to Weaviate...")

connection_attempts = [
    {"host": "localhost", "port": 8080, "grpc_port": 50051},
    {"host": "localhost", "port": 8081, "grpc_port": 50051},
    {"host": "127.0.0.1", "port": 8080, "grpc_port": 50051},
]

weaviate_client = None
for config in connection_attempts:
    try:
        print(f"  Trying {config['host']}:{config['port']}...")
        weaviate_client = weaviate.connect_to_local(
            host=config['host'],
            port=config['port'],
            grpc_port=config['grpc_port']
        )
        print(f"Connected to Weaviate at {config['host']}:{config['port']}")
        break
    except Exception as e:
        print(f"Failed: {str(e)[:50]}")
        continue

if weaviate_client is None:
    raise ConnectionError("Could not connect to Weaviate. Please start it with: podman run -d -p 8080:8080 -p 50051:50051 semitechnologies/weaviate:latest")

# Create Weaviate vector store for LlamaIndex
vector_store = WeaviateVectorStore(
    weaviate_client=weaviate_client,
    index_name="EcommerceDocuments",
    text_key="text"
)

print("✅ Weaviate vector store configured")

In [ ]:
# Load documents and create index
from llama_index.core import StorageContext

print("📚 Loading documents from docs folder...")

# Path to docs folder
docs_folder = Path("./docs")
if not docs_folder.exists():
    print(f"older not found: {docs_folder}")
    print("Please create a 'docs' folder and add your .txt files")
    raise FileNotFoundError(f"docs folder not found at {docs_folder}")

# Check for text files
txt_files = list(docs_folder.glob("*.txt"))
if not txt_files:
    print(f"No .txt files found in {docs_folder}")
    raise FileNotFoundError("No text files found in docs folder")

print(f"Found {len(txt_files)} text file(s):")
for txt_file in txt_files:
    print(f"   - {txt_file.name}")

# Load documents directly from the docs folder using SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir=str(docs_folder)).load_data()
print(f"\nLoaded {len(documents)} document(s)")

# Display document info
total_chars = sum(len(doc.text) for doc in documents)
print(f"   Total characters: {total_chars:,}")

# Parse documents into nodes with custom chunk size
print("\n🔨 Parsing documents into chunks...")
node_parser = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=50
)
nodes = node_parser.get_nodes_from_documents(documents)
print(f"Created {len(nodes)} nodes")

# Create storage context with Weaviate
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create vector store index
print("\nCreating vector store index...")
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True
)

print("\nIndex created successfully!")
print(f"   Total nodes indexed: {len(nodes)}")

In [ ]:
# Create query engine with custom settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

print("⚙️ Configuring query engine...")

# Configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,  # Retrieve top 3 most relevant chunks
)

# Configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact",  # Use compact mode for concise responses
)

# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=10,
    response_mode="compact",
    streaming=False
)

print("Query engine configured")
print(f"   Retrieval mode: Top-3 similarity search")
print(f"   Response mode: Compact")
print(f"   LLM: Circuit AI (gpt-4.1)")
print(f"   Embeddings: Ollama (mxbai-embed-large)")

In [ ]:
# Test the RAG system
import textwrap

def ask_question(question, show_sources=True):
    """
    Ask a question using LlamaIndex RAG pipeline
    
    Args:
        question (str): Your question
        show_sources (bool): Whether to display source nodes
    """
    print("\n" + "=" * 80)
    print(f"Question: {question}")
    print("=" * 80)
    
    # Query the index
    print("\nSearching knowledge base...")
    response = query_engine.query(question)
    
    # Display answer
    print("\nAnswer:")
    print("=" * 80)
    wrapped_answer = textwrap.fill(str(response), width=80)
    print(wrapped_answer)
    print("=" * 80)
    
    # Display source nodes if requested
    if show_sources and hasattr(response, 'source_nodes'):
        print("\nSource Nodes:")
        print("-" * 80)
        for i, node in enumerate(response.source_nodes, 1):
            score = node.score if hasattr(node, 'score') else 'N/A'
            print(f"\n  Source {i} (Relevance: {score}):")
            text_preview = node.text[:200] + "..." if len(node.text) > 200 else node.text
            wrapped_text = textwrap.fill(text_preview, width=76, initial_indent='    ', subsequent_indent='    ')
            print(wrapped_text)
        print("\n" + "-" * 80)
    
    return response


# Test with sample questions
print("\nLlamaIndex RAG System Ready!")
print("=" * 80)

# Run first test query
test_response = ask_question("Create Jira User Stories for Personalised Product Recommendation", show_sources=True)

## Interactive Query Interface

Use the cell below to ask questions about the documents you indexed. The query engine retrieves relevant chunks from Weaviate and synthesizes answers using the local Ollama model (`gemma:2b`).

## Advanced: Chat Engine with Conversation History

LlamaIndex supports conversational interactions with memory. Responses are generated by the local Ollama model (`gemma:2b`) and retrieval uses Weaviate. For faster iteration, consider using a smaller model.

In [ ]:
# Create a chat engine for conversational interactions
chat_engine = index.as_chat_engine(
    chat_mode="context",
    similarity_top_k=3
)

# Example conversation
response = chat_engine.chat("Pre Event Questions, Create User Stories")
print("Assistant:", response.response)
print("\n" + "="*80 + "\n")

# Follow-up question (maintains context)
response = chat_engine.chat("Also Provide Acceptance Criteria")
print("Assistant:", response.response)